# TP2: Machine Learning

In [51]:
import pandas as pd

# import ML packages
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn import linear_model
from sklearn import preprocessing

# import Plotting pckgs
from matplotlib import pyplot as plt
%matplotlib inline

In [52]:
trainingSet = pd.read_csv('../CSVs/improved_trip_train.csv')
testingSet = pd.read_csv('../CSVs/improved_trip_test.csv')
trainingSet.dtypes

id                     int64
duration               int64
start_date            object
start_dayOfWeek        int64
start_week             int64
start_quarter          int64
start_time            object
start_hour             int64
start_minute           int64
start_station_name    object
start_station_id       int64
end_date              object
end_dayOfWeek          int64
end_week               int64
end_quarter            int64
end_time              object
end_hour               int64
end_minute             int64
end_station_name      object
end_station_id         int64
bike_id                int64
subscription_type     object
zip_code              object
dtype: object

In [53]:
trainingSet.head()

,id,duration,start_date,start_dayOfWeek,start_week,start_quarter,start_time,start_hour,start_minute,start_station_name,...,end_week,end_quarter,end_time,end_hour,end_minute,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,907649,396,2015-08-27,3,35,3,08:36:00,8,36,Harry Bridges Plaza (Ferry Building),...,35,3,08:43:00,8,43,Embarcadero at Sansome,60,187,Subscriber,94602
1,384043,636,2014-07-28,0,31,3,22:06:00,22,6,Market at 10th,...,31,3,22:17:00,22,17,Washington at Kearny,46,417,Subscriber,94133
2,316176,334,2014-06-09,0,24,2,08:42:00,8,42,Market at Sansome,...,24,2,08:47:00,8,47,2nd at Folsom,62,281,Subscriber,94107
3,618874,666,2015-01-26,0,5,1,16:55:00,16,55,San Francisco Caltrain 2 (330 Townsend),...,5,1,17:07:00,17,7,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602
4,910977,318,2015-08-29,5,35,3,15:09:00,15,9,Market at 10th,...,35,3,15:14:00,15,14,Powell Street BART,39,607,Subscriber,94709


In [54]:
trainingSet.shape

(549961, 23)

In [55]:
testingSet.head()

,id,start_date,start_dayOfWeek,start_week,start_quarter,start_time,start_hour,start_minute,start_station_name,start_station_id,...,end_week,end_quarter,end_time,end_hour,end_minute,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,504737,2014-10-18,5,42,4,11:25:00,11,25,Embarcadero at Sansome,60,...,42,4,12:12:00,12,12,Powell at Post (Union Square),71,426,Customer,77009
1,530846,2014-11-05,2,45,4,13:00:00,13,0,Embarcadero at Folsom,51,...,45,4,13:09:00,13,9,Broadway St at Battery St,82,454,Subscriber,94132
2,813140,2015-06-18,3,25,2,17:34:00,17,34,San Francisco Caltrain (Townsend at 4th),70,...,25,2,17:37:00,17,37,2nd at Townsend,61,370,Subscriber,94107
3,897674,2015-08-20,3,34,3,07:06:00,7,6,Civic Center BART (7th at Market),72,...,34,3,07:15:00,7,15,Townsend at 7th,65,451,Subscriber,94582
4,322830,2014-06-13,4,24,2,08:46:00,8,46,San Francisco Caltrain 2 (330 Townsend),69,...,24,2,08:57:00,8,57,Embarcadero at Folsom,51,603,Subscriber,95014


In [56]:
trainingSet.duration.mean()

1113.4153549069842

### Let's study correlation between data

In [57]:
trainingSet.corr()['duration']

id                 -0.003133
duration            1.000000
start_dayOfWeek     0.018351
start_week          0.004002
start_quarter       0.003648
start_hour          0.002598
start_minute        0.002586
start_station_id   -0.007001
end_dayOfWeek       0.013753
end_week            0.002100
end_quarter         0.001376
end_hour            0.009626
end_minute         -0.000530
end_station_id     -0.006026
bike_id            -0.001569
Name: duration, dtype: float64

We can see that the duration is most correlated with:
    - start_dayOfWeek     0.018351
    - end_dayOfWeek       0.013753
    - end_hour            0.009626
    - start_station_id   -0.007001

In [58]:
columns = trainingSet.columns.tolist()
columns

['id',
 'duration',
 'start_date',
 'start_dayOfWeek',
 'start_week',
 'start_quarter',
 'start_time',
 'start_hour',
 'start_minute',
 'start_station_name',
 'start_station_id',
 'end_date',
 'end_dayOfWeek',
 'end_week',
 'end_quarter',
 'end_time',
 'end_hour',
 'end_minute',
 'end_station_name',
 'end_station_id',
 'bike_id',
 'subscription_type',
 'zip_code']

In [59]:
trainingSet.dtypes

id                     int64
duration               int64
start_date            object
start_dayOfWeek        int64
start_week             int64
start_quarter          int64
start_time            object
start_hour             int64
start_minute           int64
start_station_name    object
start_station_id       int64
end_date              object
end_dayOfWeek          int64
end_week               int64
end_quarter            int64
end_time              object
end_hour               int64
end_minute             int64
end_station_name      object
end_station_id         int64
bike_id                int64
subscription_type     object
zip_code              object
dtype: object

In [60]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [61]:
trainingSet.shape

(549961, 23)

In [62]:
# Train the model using the training sets
regr.fit(trainingSet[['start_dayOfWeek', 'end_dayOfWeek', 'end_hour', 'start_station_id']], trainingSet.duration)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [63]:
prediction = regr.predict(testingSet[['start_dayOfWeek', 'end_dayOfWeek', 'end_hour', 'start_station_id']])

In [64]:
prediction

array([ 1603.57291201,  1069.66886325,  1298.99586064, ...,  1784.36773406,
        1544.50655645,   596.68919218])